In [0]:
pip install lightgbm

In [0]:
pip install hyperopt

In [0]:
%pip install databricks-feature-engineering==0.13.0.1

In [0]:
%restart_python

In [0]:
import pandas as pd
import lightgbm as lgb
import pickle
import joblib
import os
from datetime import datetime, date 
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.lightgbm
from sklearn.metrics import classification_report,roc_auc_score,f1_score
import numpy as np
from mlflow.models.signature import infer_signature
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll import scope
import numpy as np
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll import scope
from databricks.feature_engineering import FeatureEngineeringClient
from databricks.feature_engineering import FeatureEngineeringClient, FeatureLookup
from sklearn.preprocessing import LabelEncoder
import json
from mlflow.tracking import MlflowClient
import requests
from sklearn.metrics import accuracy_score

**what this notebook is doing**
Model Loading

Loads a trained LightGBM model

Uses either pickle or joblib

Feature Importance Extraction

Pulls feature importance directly from the trained model

Sorts features by gain/importance

Feature Selection

Saves:

All features with importance

Top 50 most important features

Production Readiness

Top 50 features can be:

Registered in Feature Store

Used for retraining

Used for inference pipelines

In [0]:
base_4 = spark.table('ispl_databricks.model_logs.bd_500_features_sample_training').toPandas()

In [0]:
model_trainYN = 1
data_version = "base"

input_dir = "/Volumes/ispl_databricks/default/training/ff_bd/input_dir"
output_dir = "/Volumes/ispl_databricks/default/training/ff_bd/output_dir_same_columns"
model_dir = "/Volumes/ispl_databricks/default/training/ff_bd/model_dir"

# Set data version
data_version = os.path.join("base")   # change here
# data_version = os.path.join("data_v5_new/top_20")   # alternative version

# Version-specific directories
input_dir_version = os.path.join(input_dir, data_version)
output_dir_version = os.path.join(output_dir, data_version)
model_dir_version = os.path.join(model_dir, data_version)

# Create directories if they do not exist
for directory in [output_dir_version, model_dir_version]:
    os.makedirs(directory, exist_ok=True)

# Model and feature file names
model_file_name = "lgb_model.pickle"
feature_file_name = "model_input_feature.pickle"


In [0]:
# Load trained LightGBM model using pickle
with open(
    os.path.join(output_dir_version, model_file_name), "rb"
) as f:
    model = pickle.load(f)
# Alternatively load model using joblib
# (Preferred for sklearn / LightGBM models)
model = joblib.load(
    os.path.join(output_dir_version, "job_model.pkl")
)

In [0]:
# Extract feature importance from the model
feature_imp_df = pd.DataFrame({'feature_name':model.feature_name_,'importance':model.feature_importances_})
# Sort features by importance (highest first)
feature_imp_df.sort_values(by=["importance"],ascending=False,inplace=True)
# Save full list of features with importance
top_n_features = feature_imp_df.loc[:,'feature_name'].tolist()

feature_imp_df.to_csv(os.path.join(output_dir_version,"all_gain_feautures.csv"))
# Select top 50 most important features
top_n_features = feature_imp_df.iloc[0:50]

# --------------------------------------------
# Save top 50 features for downstream use
# (feature store / retraining / inference)
# -------------------------------------------
top_n_features.to_csv(os.path.join(output_dir_version,"top50_gain_feautures.csv"))

In [0]:
# --------------------------------------------
# Extract only the feature names from the
# top N features DataFrame
# ----------------------------------------
feature_to_store = top_n_features["feature_name"].tolist()
# Add the primary key (loan_id)
# This is required for joins, feature store,
# training, and inference consistency
feature_to_store = feature_to_store + ['loan_id']
# Select only the required columns from base data
# This creates the final feature store DataFrame
# containing:
#   - Top N important features
#   - Primary key (loan_id)
feature_store_df = base_4.loc[:,feature_to_store]

In [0]:
# just for precaution if any column is object type or category can eb xonvert to int except loan_id
from sklearn.preprocessing import LabelEncoder

label_encoders = {}

for col in feature_store_df.columns:
    dtype = feature_store_df[col].dtype

    if col == "loan_id":
        continue

    if dtype.name in ["category", "object"]:
        le = LabelEncoder()
        feature_store_df[col] = le.fit_transform(
            feature_store_df[col].astype(str)
        )
        label_encoders[col] = le


In [0]:
feature_store_df  = spark.createDataFrame(feature_store_df)
feature_store_df = feature_store_df.drop_duplicates(['loan_id'])

In [0]:
display(feature_store_df)

In [0]:
feature_store_df.dtypes

In [0]:
# creation of feature store to store features that we will use to train top 50 features model
fe = FeatureEngineeringClient()
fe.create_table(
  name="ispl_databricks.model_logs.bd_final_feature_stores",
  primary_keys=["loan_id"],
  df=feature_store_df,
  description="Feature table for the bank"
)
fe.write_table(
    name="ispl_databricks.model_logs.bd_final_feature_stores",
    df=feature_store_df,                 # Spark or pandas DataFrame
    mode="merge"           # works like upsert (recommended)
)